In [1]:
#importa as bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from functools import reduce

In [2]:
#Abre o Fase atual
df = pd.read_excel("C:\\Users\\WIN10\\Documents\\Mont seguro\\Relatorio_Comercial\\Fase Atual.xlsx")

#Abre o Ganhos
df2 = pd.read_excel("C:\\Users\\WIN10\\Documents\\Mont seguro\\Relatorio_Comercial\\Ganhos.xlsx")

#Abre o Negociação
df3 = pd.read_csv("C:\\Users\\WIN10\\Documents\\Mont seguro\\Automação\\Relatorio_Comercial\\dados\\Negociação.csv", encoding='latin-1')

In [3]:
def preprocess_dataframe(df):
    # cria um objeto que convete string em datetime
    def extract_month(date_string):
        date_obj = datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f')
        return date_obj.month

    # cria um objeto que convete string em datetime
    def extract_year(date_string):
        date_obj = datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f')
        return date_obj.year

    # cria uma nova coluna 'numero do mês' baseada na coluna 'Previsão de Pagamento'
    df['numero do mês'] = df['Previsão de Pagamento'].dropna().apply(extract_month)
    # cria uma nova coluna 'numero do ano' baseada na coluna 'Previsão de Pagamento'
    df['numero do ano'] = df['Previsão de Pagamento'].dropna().apply(extract_year)

    #transforma numeros nan em -1 inteiro
    df['numero do mês'] = np.nan_to_num(df['numero do mês'], nan=-1).astype(int)

    df['numero do ano'] = np.nan_to_num(df['numero do ano'], nan=-1).astype(int)
    
    #tira o primeiro ponto dos numero, depois troca a virgula por ponto e transforma string em float
    if df['Valor'].dtype == 'object':
        df['Valor'] = df['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)

    
    return df

In [4]:
#aplica as modificações nos DataFrames

df = preprocess_dataframe(df)
df2 = preprocess_dataframe(df2)
df3 = preprocess_dataframe(df3)


C:\Users\WIN10\AppData\Local\Temp\ipykernel_18316\3509542288.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Valor'] = df['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)
C:\Users\WIN10\AppData\Local\Temp\ipykernel_18316\3509542288.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Valor'] = df['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)


In [5]:
#cria filtro previsão proximo mes
filtro_proximo = ((df['numero do mês'] == datetime.datetime.now().month +1) & 
          (df['Fase atual'].isin(['Análise', 'Pagamento'])))



# cria filtro previsão esse mês
filtro_mes = ((df['numero do mês'] == datetime.datetime.now().month) | 
              (df['Previsão de Pagamento'].isna() &
               df['Fase atual'].isin(['Análise', 'Pagamento'])))


#cria filtro implantados
filtro_implantados = ((df2['numero do mês'] == datetime.datetime.now().month ) & 
                      (df2['numero do ano'] == datetime.datetime.now().year ))


#cria filtro pagamento

filtro_pagamentos = (df['Fase atual'].isin(['Pagamento']))



#cria filtro analise
filtro_analise = (df['Fase atual'].isin(['Análise']))


#cria filtro Documentação
filtro_documentacao = (df['Fase atual'].isin(['Documentação']))

#cria filtro fallow-up
filtro_follow_up = (df['Fase atual'].isin(['Follow Up']))

#cria filtro Documentação para filtros
filtro_documentacao2 = (df['Fase atual'].isin(['Documentação']))

In [6]:
newdf1 = filtro_proximo.to_frame(name='previsão proximo mes')
newdf2 = filtro_mes.to_frame(name='previsão esse mes')
newdf3 = filtro_implantados.to_frame(name='implantados')
newdf4 = filtro_pagamentos.to_frame(name='pagamento')
newdf5 = filtro_analise.to_frame(name='analise')
newdf6 = filtro_documentacao2.to_frame(name='Documentação')
newdf7 = filtro_follow_up.to_frame(name='fallow-up')


In [7]:
df3['contagem'] = df3['Responsável']
df['contagem'] = df['Responsável']

In [8]:
def agrupa_e_soma(df, filtro, coluna_valor, nome_coluna):
    df_agrupado = df[filtro].groupby(['Responsável'])[coluna_valor].sum().reset_index()
    df_agrupado = df_agrupado.rename(columns={coluna_valor: nome_coluna})
    return df_agrupado

df_responsavel1 = agrupa_e_soma(df, filtro_proximo, 'Valor', 'previsão proximo mês')
df_responsavel2 = agrupa_e_soma(df, filtro_mes, 'Valor', 'previsão esse mês')
df_responsavel3 = agrupa_e_soma(df2, filtro_implantados, 'Valor', 'implantados')
df_responsavel4 = agrupa_e_soma(df, filtro_pagamentos, 'Valor', 'pagamento')
df_responsavel5 = agrupa_e_soma(df, filtro_analise, 'Valor', 'análise')
df_responsavel6 = agrupa_e_soma(df, filtro_documentacao, 'Valor', 'documentação')
df_responsavel7 = df[filtro_follow_up].groupby(['Responsável'])['contagem'].count().reset_index().rename(columns={'contagem': 'Follow Up'})
df_responsavel8 = df3.groupby(['Responsável'])['contagem'].count().reset_index().rename(columns={'contagem': 'Em Negociação'})

In [9]:
#concatena todos os dataframes em um unico dataframe
dataframes = [df_responsavel2, df_responsavel3, df_responsavel4,
              df_responsavel5, df_responsavel6, df_responsavel7, df_responsavel8, df_responsavel1]

df_resultado = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, on='Responsável', how='left'),
                      dataframes)

## -----------------------------------------------Ultima linha-----------------------------------------------------

In [10]:
dataframes = [newdf1, newdf2, newdf3, newdf4,
              newdf5, newdf6, newdf7]

df_filtros = pd.concat(dataframes, axis=1)

## -----------------------------------------------Ultima linha-----------------------------------------------------

In [11]:
def salvar_dataframe(dataframe, nome_arquivo):
    # Define o caminho onde o arquivo será salvo
    caminho_salvar = 'C:\\Users\\WIN10\\Documents\\Mont seguro\\Relatorio_Comercial'
    # Define o nome completo do arquivo, incluindo o caminho
    caminho_completo = os.path.join(caminho_salvar, nome_arquivo)
    # Salva o DataFrame em um arquivo Excel
    dataframe.to_excel(caminho_completo, index=False)

In [12]:
salvar_dataframe(df_resultado, 'Negociação.xlsx')

In [13]:
salvar_dataframe(df_filtros, 'Filtros.xlsx')

In [14]:
df_resultado

,Responsável,previsão esse mês,implantados,pagamento,análise,documentação,Follow Up,Em Negociação,previsão proximo mês
0,Giovanni,14438.79,4248.94,3443.98,7724.83,8940.05,15,28,NaN
1,Hiltom,9055.41,8872.73,560.66,6925.67,4513.36,9,42,560.66
2,Vitória Monteiro,10325.96,781.56,4199.35,9082.22,4916.68,7,50,4978.75
